# Complete Usage
___

在这个示例中，将尽可能详细的介绍各项配置、预置 agent、组织架构等内容。

相较于 Quick Start，当前 notebook 有些冗长，建议初次接触者可以先从 Quick Start 开始。

## 安装框架
___

AutoGan requires Python version >= 3.8. It can be installed from pip:

In [1]:
# %%capture --no-stderr
# %pip install --upgrade pyautogan

In [2]:
import autogan

# print(autogan.__file__)

## 构建 Agents
___

所有 agent 都是在 UniversalAgent 基类之上构建的，其主要属性如下：

* **name:** 该属性相当于 agent 的地址，用于 agent 之间的相互识别与沟通，因此该属性的值应当是全局唯一的。
* **duty:** 用于向其他 agent 介绍自身职责及能力。
* **work_flow:** 用于定义 agent 自身的工作流程。


### 人工 agent

真人员工的数字化身，用来与其他 agent 协同工作。

In [3]:
# 参数：
# 1.agent's name 应当是唯一的
# 2.向其他 agent 介绍自己的职责
human = autogan.HumanAgent("客户", "please help me.")

### 普通 agent

仅依靠 LLM 本身的能力完成各项任务，不使用其他工具函数辅助。

1. 配置

In [4]:
# 通过 dict_from_json 函数读取 LLM 配置
llm_config_dict = autogan.dict_from_json("LLM_CONFIG")

    dict_from_json 该函数会优先从环境变量 "LLM_CONFIG" 中获取配置，如环境变量中没有找到 "LLM_CONFIG"，则会从当前目录中名为 "LLM_CONFIG" 的文件内读取配置。
    
    配置内容解构如下：
    ```json
    {
        "main_model": {
            "api_key_list": [
                {
                    "model": "gpt-4",
                    "api_key": "<your OpenAI API key here>",
                    "api_type": "openai"
                },{
                    "model": "gpt-4-32k",
                    "api_key": "<your Azure OpenAI API key here>",
                    "api_base": "<your Azure OpenAI API base here>",
                    "api_type": "azure",
                    "api_version": "2023-07-01-preview"
                },{
                    "model": "gpt-4",
                    "Authorization": "<your OpenAI Like API header Authorization here>",
                    "api_type": "openai like",
                    "url": "<your Azure OpenAI API url here>"
                }
            ],
            "model_filter": "gpt-4",
            "max_messages_tokens": 4096
        },
        "summary_model": {
            "api_key_list": [
                {
                    "model": "gpt-3.5-turbo",
                    "api_key": "<your OpenAI API key here>",
                    "api_type": "openai"
                }
            ],
            "model_filter": "gpt-4",
            "max_messages_tokens": 4096
        },
        "request_interval_time": 1,
        "request_timeout": 120,
        "max_retries": 3
    }
    ```
    
    - main_model: The LLM configuration of the agent's main body.
        - api_key_list: LLM model configuration list. During the request process, the system will attempt in a predetermined order. If the current request fails, the system will automatically switch to the next configuration for another attempt.
        - max_messages_tokens: LLM model configuration used for compressing future conversation records or other text content.
    
    - summary_model: The LLM configuration used for compressing context and generating text summaries. This configuration is optional, when it is missing, the framework will use 
        the configuration of the main_model for context or text compression.
        - api_key_list: LLM model configuration list. During the request process, the system will attempt in a predetermined order. If the current request fails, the system will automatically switch to the next configuration for another attempt.
        - max_messages_tokens: LLM model configuration used for compressing future conversation records or other text content.
    
    - request_interval_time: The interval time of LLM requests.
    
    - request_timeout:The timeout of LLM requests.
    
    - max_retries: The maximum number of retries for LLM requests.
    
    **注意：**  框架目前仅适配了 openai、azure、自定义类 openai 接口的 chat 模型。

2. 构建

In [5]:
# 参数：
# 1.agent's name 应当是唯一的
# 2.向其他 agent 介绍自己的职责
# 3.agent's workflow
cust_manager = autogan.UniversalAgent("客户经理", duty="Responsible for receiving customers", work_flow="""
1. Do everything possible to meet the legal needs of the client, do not give up before satisfying the client's requirements. Note: If the client provides documents, consider finding the answers from the documents first.
2. If there is no correct answer, make it clear, instead of giving a wrong answer.""")

# 参数：
# 1.agent's name 应当是唯一的
# 2.向其他 agent 介绍自己的职责
# 3.agent's workflow
coder = autogan.UniversalAgent("Coder", duty="I can accomplish tasks previously impossible for you by writing programs.", work_flow="I hope you are an experienced Python programmer. Before writing code, please prepare the following: If code is generated, you must @CodeExecSpec at the beginning of the reply content. The output code must be enclosed with ``` symbol. When the program runs with missing dependencies, you can write the statement to install the dependencies and then @CodeExecSpec to install the relevant dependencies.")

### 网络搜索 agent

ToolAgentSearch 为框架预置的 agent，可以通过一下方式进行网络搜索：

* 利用 Google Programmable Search Engine 的 Custom Search JSON API 从网络上搜索资料。
  
* 利用 WolframAlpha API 获取问题的答案。

1. 配置

* 获取 Custom Search JSON API 配置参数，请参见其 [官方文档](https://developers.google.com/custom-search/v1/overview?hl=en)

* 获取 WolframAlpha API 配置参数，请访问其 官方网站 创建 APP ID

In [6]:
# 通过 dict_from_json 函数读取 google search api 配置
search_config_dict = autogan.dict_from_json("SEARCH_CONFIG")

    dict_from_json 该函数会优先从环境变量 "SEARCH_CONFIG" 中获取配置，如环境变量中没有找到 "SEARCH_CONFIG"，则会从当前目录中名为 "SEARCH_CONFIG" 的文件内读取配置。
        
    配置内容解构如下：
    ```json
    {
        "google_search": {
            "cx": "<your Programmable Search Engine ID here>",
            "key": "<your Custom Search JSON API access key here>"
        },
        "wolfram_alpha": {
            "app_id": "<your WolframAlpha APP ID here>"
        }
    }
    ```
2. 构建

In [7]:
# 参数：
# 1.Custom Search JSON API 配置
# 2.agent's name 应当是唯一的
web_search_exp = autogan.ToolAgentSearch(search_config_dict, name="WebSearchExp")

**提示：**你可以通过初始化参数或属性对其 duty 和 workflow 进行自定义。

### Mail agent

ToolAgentMail 为框架预置的 agent，作用是根据要求发送邮件。

1. 配置

In [8]:
mail_config_dict = autogan.dict_from_json("MAIL_CONFIG")

    dict_from_json 该函数会优先从环境变量 "MAIL_CONFIG" 中获取配置，如环境变量中没有找到 "MAIL_CONFIG"，则会从当前目录中名为 "MAIL_CONFIG" 的文件内读取配置。
        
    配置内容解构如下：
    ```json
    {
        "server": "<your Mail Send Server here>",
        "port": 465,
        "username": "<your Mail Server User Name here>",
        "password": "<your Mail Server User Password here>"
    }
    ```
   
2. 构建

In [9]:
# 参数：
# 1.Mail 配置
# 2.agent's name 应当是唯一的
mail_spec = autogan.ToolAgentMail(mail_config_dict, "MailSpec")

**提示：**你可以通过初始化参数或属性对其 duty 和 workflow 进行自定义。

### 文档操作 agent

ToolAgentFile 为框架预置的 agent，可打开 word、excel、pdf 文档，以及向 word 文档中追加文本内容。

In [10]:
# 参数：
# 1.agent's name 应当是唯一的
file_exp = autogan.ToolAgentFile(name="FileExp")

**提示：**你可以通过初始化参数或属性对其 duty 和 workflow 进行自定义。

### 代码执行 agent

ToolAgentCodeExecution 为框架预置的 agent，可以执行 Python 和 shell 代码并返回结果。

In [11]:
# 参数：
# 1.agent's name 应当是唯一的
code_execution_agent = autogan.ToolAgentCodeExecution("CodeExecSpec")

**提示：**你可以通过初始化参数或属性对其 duty 和 workflow 进行自定义。

## 构建组织架构
___

考虑到并非所有 agent 之间都需要建立协作关系，且过多无用的协作关系会影响到 agent 决策的准确性。
因此本框架引入了组织架构的概念，让需要相互协作的 agent 组成独立的数字部门，以更加专注于解决特定领域的问题。

创建组织架构的方式非常简单，只需要定义一个存储 agent 对象的多维列表。

In [12]:
org_structure = [human, cust_manager, web_search_exp, mail_spec, [coder, code_execution_agent], file_exp]

每个 list 相当于一个部门，各部门的第一个 agent 为部门的 leader，负责与上级部门的 agent 进行沟通。
因此 leader 的 duty 属性应当概述整个部门的职测与能力。

## 开始会话
___

与 agent 的初次对话，需要在发送的消息中添加 task_tag 符号，用于创建任务，task_tag 的默认值为 "/task"。

后续对话中如果使用 task_tag 符号，则是对当前任务发表意见或看法。

In [13]:
# 参数：
# 1.组织架构
# 2.邀请第一个发言的 agent 名称
# 3.更改 task_tag 的值，默认为 "/task"
autogan.Switch(llm_config_dict, org_structure, "hello", human);

Please enter:  @客户经理 hello 你好



客户: @客户经理 hello 你好


>>>>>>>> Observe whether the previous conversation fell into a cycle:

客户经理: 
None.


>>>>>>>> Reflect on whether there are any errors in the previous dialogue process:

客户经理: 
Step 5: Analysis Results

Whether there are unresolved errors in the previous conversation: No

If there are unresolved errors: N/A

If there are unresolved errors, the root cause of the error: N/A

If there are unresolved errors, how to solve it next: N/A

Analysis: Based on the given conversation, there are no unresolved errors. The conversation is a simple task request from the client, and there is no specific error or issue mentioned. Therefore, there is no need for further analysis or resolution.


>>>>>>>> Think about what to do next:


KeyboardInterrupt: 